In [92]:
import ccxt
import pandas as pd 
from tqdm import tqdm

In [84]:
def get_data(exchange, s, date): 
    timeframe = '1m'
    bars = 100000
    # since = exchange.parse8601(f'{date}T00:00:00Z')
    since = exchange.parse8601(date)
    ohlcv = exchange.fetch_ohlcv(s, timeframe, since, bars)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [40]:
pd.concat(dfs).index[-1]

Timestamp('2021-01-01 16:39:00')

In [19]:
exchange = ccxt.binanceus()

# extract all symbols supported by the exchange
exchange.load_markets()
symbols = exchange.symbols

In [70]:
# example: 
dfs = []
for s in symbols[:10]: 
    try: 
        df = get_data(exchange, s, '2021-01-12')
        df["symbol"] = s
        dfs.append(df)
    except: 
        pass

In [78]:
dfs[-1].index[-1].year

2021

Note, I think limit is around 999? so need to get separately and concate together 

In [89]:
def get_data_per_symbol(s): 
    start = "2022-01-01T00:00:00Z"
    dfs = []
    try: 
        df = get_data(exchange, s, start)
    except: 
        return []
    last_date = df.index[-1] 
    dfs.append(df)
    while last_date.year == 2022: 
        time = pd.concat(dfs).index[-1]
        time = time.strftime("%Y-%m-%dT%H:%M:%SZ")
        try: 
            df = get_data(exchange, s, time)
        except:
            break
        last_date = df.index[-1]
        dfs.append(df)
    df = pd.concat(dfs)
    df["symbol"] = s
    return df

In [90]:
# example: 
get_data_per_symbol("BTC/USD")

,open,high,low,close,volume,symbol
timestamp,,,,,,
2022-01-01 00:00:00,46212.06,46266.73,46208.24,46245.79,0.354754,BTC/USD
2022-01-01 00:01:00,46233.95,46323.35,46225.56,46303.73,0.393746,BTC/USD
2022-01-01 00:02:00,46312.26,46370.48,46305.75,46370.21,0.846843,BTC/USD
2022-01-01 00:03:00,46358.23,46379.75,46309.69,46333.78,0.313366,BTC/USD
2022-01-01 00:04:00,46328.07,46331.98,46300.00,46322.71,0.124923,BTC/USD
...,...,...,...,...,...,...
2023-01-01 15:03:00,16540.30,16540.91,16540.30,16540.91,0.037744,BTC/USD
2023-01-01 15:04:00,16540.91,16540.91,16538.53,16540.28,0.840938,BTC/USD
2023-01-01 15:05:00,16540.22,16540.22,16539.65,16539.65,0.022240,BTC/USD


In [94]:
for s in tqdm(symbols): 
    df = get_data_per_symbol(s) 
    if len(df) == 0: 
        continue
    base = s.split("/")[0]
    quote = s.split("/")[1]
    df.to_csv(f"data/{base}_{quote}.csv")

  1%|          | 5/541 [02:49<5:08:53, 34.58s/it]